<a href="https://colab.research.google.com/github/swilsonmfc/linear/blob/main/Assumptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression
![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Anscombe%27s_quartet_3.svg/1200px-Anscombe%27s_quartet_3.svg.png)

# Setup

In [2]:
import math
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling

from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from yellowbrick.regressor import ResidualsPlot
from yellowbrick.regressor import PredictionError

import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.graphics.regressionplots as regplot
from statsmodels.stats.outliers_influence import variance_inflation_factor

from patsy import dmatrices

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data

In [3]:
boston = load_boston()
print(boston['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [4]:
bostonDF = pd.DataFrame(boston['data'], columns=boston['feature_names'])
bostonDF['TARGET'] = boston['target']
bostonDF.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Categorical

In [5]:
def ageBin(age):
    if age < 30: return 'NEW'
    if age < 60: return 'USED'
    if age < 80: return 'OLD'
    if age < 95: return 'CREAKY'
    return 'FOSSIL'
bostonDF['AGE_BIN'] = bostonDF['AGE'].apply(ageBin)
bostonDF['AGE_BIN'].hist()

# Preprocessing

In [6]:
bostonDF['AGE_BIN'] = bostonDF['AGE_BIN'].astype('category')
bostonDF = bostonDF.drop('AGE', axis=1)

# Model

In [7]:
df = bostonDF.copy()
y = df['TARGET']
X = df.drop('TARGET', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [8]:
tDF = pd.concat([X_train, y_train], axis=1)
f = 'TARGET ~ CRIM + ZN + INDUS + CHAS + NOX + RM + DIS + RAD + TAX + PTRATIO + B + LSTAT + AGE_BIN'
lm = smf.ols(formula=f, data=tDF).fit()

# Assumptions & Diagnostics

## L (inear)
* Data should be visibly linear - hard to do in high dimensional spaces
* You can attempt to fit polynomial independent variables or apply other transformation to capture / fit the non-linerity
![](https://study.com/cimages/multimages/16/linear_function_vs_nonlinear_function.png)

## I (ndependent)
* Each data point should be independent. 
* We want to avoid colinearity
 * When we have high correlation on two features, there is little information from both in the model
 * Consider dropping out one of the correlated features
* Example violations of this assumption 
 * Include time series (which can introduce autocorrelation)
 * Height regression that includes height of left leg and height of right leg in model

### Variable Inflation Factor
Variance Inflation Factor (VIF) measures the colinearity of predictor variables. VIF is the ratio of the variance of all model's coefficients divided by the variance of a single coefficient if it were fit in isolation.  

In [9]:
variables = lm.model.exog
vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]

vifDF = pd.DataFrame()
vifDF['Coefficient'] = lm.params
vifDF['VIF'] = vif
vifDF['Flag'] = np.where(vifDF.VIF > 5, True, False)
vifDF = vifDF.drop('Intercept', axis=0)
vifDF

,Coefficient,VIF,Flag
AGE_BIN[T.FOSSIL],1.247437,1.702363,False
AGE_BIN[T.NEW],0.796080,2.496762,False
AGE_BIN[T.OLD],-0.785411,1.662618,False
AGE_BIN[T.USED],0.353033,2.657472,False
CRIM,-0.103085,1.827122,False
ZN,0.022089,2.440392,False
INDUS,-0.022603,4.387071,False
CHAS,2.360390,1.085797,False
NOX,-16.448766,4.419138,False
RM,4.312501,1.926968,False


### Correlation RAD & TAX
* These two features are highly correlated and score high on the VIF.
 * RAD = index of accessibility to radial highways
 * TAX = full-value property-tax rate per $10,000

In [10]:
print(np.corrcoef(df.TAX, df.RAD))

[[1.         0.91022819]
 [0.91022819 1.        ]]


### Corrections
* Consider removing one of the highly correlated values
* Cost of collecting or maintaining the feature may be instructive

## N (ormally Distributed Residuals)
* Residuals should be cloud shaped with no discernable pattern
* Check highly influential points (possible outliers)
* Consider transformations to independent and dependent variables

### Residual Plot

In [11]:
preds = lm.predict(X_train)
plt.scatter(preds, -lm.resid)
plt.title('Residual Plot (Residuals vs Pred)')
plt.ylabel('Residual')

Text(44.125, 0.5, 'Residual')

### Jarque Bera Test for Normality
* Goodness of fit test
* Does the sample data have skewness & kurtosis of a normal distribution
* The statistic is always positive, and when far from zero, it signals non-normality

In [12]:
cols = ['Jarque-Bera', 'Chi^2 two-tail prob.', 'Skew', 'Kurtosis']
test = sms.jarque_bera(lm.resid)
pd.DataFrame(zip(cols, test)).T

,0,1,2,3
0,Jarque-Bera,Chi^2 two-tail prob.,Skew,Kurtosis
1,632.62,4.24927e-138,1.47013,8.37925


### Leverage & Influence Plot
Leverage
* The ability to exert influence over the model
* Measures of how far an observation on the predictor variable is from from the mean of the predictor. 
* The higher the leverage value, the more potential it has to impact the fitted model.

Influence
* Degree to which the data point affects the model. 
* Measures the effect of a point on the regression model's predictions.
* A high influence point usually has leverage.

Watch for:
* High leverage and large residual - No points
* High Leverage and small residual - Point (380)
* Low Leverage and large residual - Points (372, 368, 364)

In [13]:
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.influence_plot(lm, ax=ax, criterion="cooks")

### High Leverage Points

In [14]:
points = [380, 418]
X_train[X_train.index.isin(points)]

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT,AGE_BIN
418,73.5341,0.0,18.1,0.0,0.679,5.957,1.8026,24.0,666.0,20.2,16.45,20.62,FOSSIL
380,88.9762,0.0,18.1,0.0,0.671,6.968,1.4165,24.0,666.0,20.2,396.90,17.21,CREAKY


### Large Residual Points

In [15]:
points = [364, 372, 368]
X_train[X_train.index.isin(points)]

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT,AGE_BIN
364,3.47428,0.0,18.1,1.0,0.718,8.780,1.9047,24.0,666.0,20.2,354.55,5.29,CREAKY
368,4.89822,0.0,18.1,0.0,0.631,4.970,1.3325,24.0,666.0,20.2,375.52,3.26,FOSSIL
372,8.26725,0.0,18.1,1.0,0.668,5.875,1.1296,24.0,666.0,20.2,347.88,8.88,CREAKY


In [16]:
pred = lm.predict(X_train)
plt.title('Large Residual Points (Predicted vs Actual)')
plt.scatter(y_train, pred)
plt.ylabel('Actual')
for pt in points:
    plt.scatter(pred.loc[pt], y_train.loc[pt], color='red')

In [17]:
preds = lm.predict(X_train)
plt.scatter(preds, -lm.resid)
plt.ylabel('Residual')
plt.title('Predicted vs Residual')
for pt in points:
    plt.scatter(pred.loc[pt], -lm.resid[pt], color='red')

## E (rror Constant)
* Homoskedasticity vs Heteroskedasticity
 * Example - As income goes up our predicted error increases
 * Example - As we play more minutes of basketball our predicted scoring error increases
* Typically spot with visual plot of residual vs predicted value (look for cone shape)
* If find heteroskedasticity we typically:
 * Adjust the model - add / drop featurews
 * Box Cox (log / sqrt / power) transforms on features and the target

![](https://www.albert.io/blog/wp-content/uploads/2016/11/heteroscedastic-relationships.png)

### Visual Inspection
* There is some structure to the residual plot. 
* Indicative of heteroksedaticity

In [18]:
preds = lm.predict(X_train)
plt.scatter(preds, -lm.resid)
plt.ylabel('Residual')
plt.title('Predicted vs Residual')
plt.show()

### Breusch Pagan Test
Produces a p-value that when less that a significance level of 0.05, therefore we can reject the null hypothesis that the variance of the residuals is constant and infer that heteroscedasticity is present.

In [19]:
cols = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
test = sms.het_breuschpagan(lm.resid, lm.model.exog)
pd.DataFrame(zip(cols, test)).T

,0,1,2,3
0,Lagrange multiplier statistic,p-value,f-value,f p-value
1,61.9112,2.4857e-07,4.37745,5.99602e-08


### Heteroscedasticity
* Heteroscedasticity doesn't cause bias in coefficients, but it makes them less precise
* Less precise estimates of coefficients are more likely to be farther from a correct population value
* Heteroscedasticity can produce p-values that are smaller than they should be.  OLS can't detect this, and in more pronounced / extreme cases could lead you to conclude a coefficient is significant when it's not.
* Ways to fix / minimize:
  * Redefine Variables - Cross-sectional data changes (i.e. Population vs Per-Capita)
  * Weighted Regression - Weight per observation, assign lower weights to higher values (inverse -- 1 / value -- is typical)
  * Transformations - Convert your dependent variable (log or BoxCox)